# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [342]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [345]:
# Load dataset 
df = pd.read_csv("data/AviationData.csv",encoding='windows-1252',low_memory=False)

# Preview the data
print(df.head())

# General info
print("\n--- DataFrame Info ---")
df.info()

# Check for NaNs
print("\n--- Missing Values ---")
print(df.isna().sum())

# Summary statistics
print("\n--- Summary Statistics ---")
print(df.describe(include='all'))


         Event.Id Investigation.Type Accident.Number  Event.Date  \
0  20001218X45444           Accident      SEA87LA080  1948-10-24   
1  20001218X45447           Accident      LAX94LA336  1962-07-19   
2  20061025X01555           Accident      NYC07LA005  1974-08-30   
3  20001218X45448           Accident      LAX96LA321  1977-06-19   
4  20041105X01764           Accident      CHI79FA064  1979-08-02   

          Location        Country   Latitude   Longitude Airport.Code  \
0  MOOSE CREEK, ID  United States        NaN         NaN          NaN   
1   BRIDGEPORT, CA  United States        NaN         NaN          NaN   
2    Saltville, VA  United States  36.922223  -81.878056          NaN   
3       EUREKA, CA  United States        NaN         NaN          NaN   
4       Canton, OH  United States        NaN         NaN          NaN   

  Airport.Name  ... Purpose.of.flight Air.carrier Total.Fatal.Injuries  \
0          NaN  ...          Personal         NaN                  2.0   
1   

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [349]:
# Look at unique values to understand filtering needs
print("Aircraft.Category:", df['Aircraft.Category'].unique())
print("Amateur.Built:", df['Amateur.Built'].unique())
print("Event.Date sample:", df['Event.Date'].head())

print(df[['Aircraft.Category', 'Amateur.Built', 'Event.Date']].isna().sum())
df = df[df['Aircraft.Category'].str.strip().str.lower() == 'airplane']
df = df[df['Amateur.Built'].str.strip().str.lower() == 'no']
# Convert Event.Date to datetime
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')

# Drop rows with invalid or missing dates
df = df.dropna(subset=['Event.Date'])

# Filter to 1983 and later
df = df[df['Event.Date'].dt.year >= 1983]
print("Remaining records:", len(df))
df.info()



Aircraft.Category: [nan 'Airplane' 'Helicopter' 'Glider' 'Balloon' 'Gyrocraft' 'Ultralight'
 'Unknown' 'Blimp' 'Powered-Lift' 'Weight-Shift' 'Powered Parachute'
 'Rocket' 'WSFT' 'UNK' 'ULTR']
Amateur.Built: ['No' 'Yes' nan]
Event.Date sample: 0    1948-10-24
1    1962-07-19
2    1974-08-30
3    1977-06-19
4    1979-08-02
Name: Event.Date, dtype: object
Aircraft.Category    56602
Amateur.Built          102
Event.Date               0
dtype: int64
Remaining records: 21447
<class 'pandas.core.frame.DataFrame'>
Index: 21447 entries, 4149 to 88886
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                21447 non-null  object        
 1   Investigation.Type      21447 non-null  object        
 2   Accident.Number         21447 non-null  object        
 3   Event.Date              21447 non-null  datetime64[ns]
 4   Location                21441 non-null  object        

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [352]:
# First, inspect relevant injury columns
injury_cols = ['Total.Fatal.Injuries', 'Total.Serious.Injuries', 
               'Total.Minor.Injuries', 'Total.Uninjured']

print(df[injury_cols].isna().sum())

# Fill NaNs with 0s since missing values likely mean "no injuries recorded"
df[injury_cols] = df[injury_cols].fillna(0)

# Ensure they are numeric (some may be strings due to original data issues)
for col in injury_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)



# Sum of all injuries + uninjured passengers = total onboard
df['Total_Onboard'] = (df['Total.Fatal.Injuries'] + df['Total.Serious.Injuries'] +
                       df['Total.Minor.Injuries'] + df['Total.Uninjured'])

# Filter out edge cases (e.g., 0 total onboard is unusable)
df = df[df['Total_Onboard'] > 0]


#  Construct Fatal/Serious Injury Rate


df['Severe_Injury_Rate'] = (
    (df['Total.Fatal.Injuries'] + df['Total.Serious.Injuries']) / df['Total_Onboard']
)


Total.Fatal.Injuries      2750
Total.Serious.Injuries    2828
Total.Minor.Injuries      2544
Total.Uninjured            711
dtype: int64


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [354]:
# Clean the Aircraft.damage column: strip whitespace and convert to lowercase
df['Aircraft.damage'] = df['Aircraft.damage'].str.strip().str.lower()

# Create a binary column: 1 if aircraft was destroyed, else 0
df['Destroyed'] = df['Aircraft.damage'].apply(lambda x: 1 if x == 'destroyed' else 0)

# Describe the key metrics
print(df[['Total_Onboard', 'Severe_Injury_Rate', 'Destroyed']].describe())


       Total_Onboard  Severe_Injury_Rate     Destroyed
count   20543.000000        20543.000000  20543.000000
mean        8.970404            0.283971      0.110403
std        36.645481            0.431704      0.313399
min         1.000000            0.000000      0.000000
25%         1.000000            0.000000      0.000000
50%         2.000000            0.000000      0.000000
75%         2.000000            0.800000      0.000000
max       588.000000            1.000000      1.000000


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [356]:
# Check unique values and missing data
print("Unique Makes:", df['Make'].nunique())
print("Top 10 Makes:\n", df['Make'].value_counts().head(10))
print("Missing values in Make:", df['Make'].isna().sum())


Unique Makes: 1309
Top 10 Makes:
 Make
CESSNA                4794
PIPER                 2777
Cessna                2270
Piper                 1183
BEECH                 1003
BOEING                 642
Beech                  409
MOONEY                 235
CIRRUS DESIGN CORP     218
AIR TRACTOR INC        217
Name: count, dtype: int64
Missing values in Make: 2


### Cleaning Tasks for the `Make` Column

-  Strip leading/trailing spaces and convert to uppercase for consistency.
-  Handle any clearly duplicated or inconsistent naming (e.g., "BOEING", "BOEING CO", "BOEING INC").
-  Remove missing values, as we cannot attribute safety metrics to unknown makes.
-  Filter out makes that occur too infrequently for reliable analysis (e.g., fewer than 50 records).


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [359]:
# Check and drop NaNs
print("Missing values in Model:", df['Model'].isna().sum())

# Drop rows with missing Model values
df = df.dropna(subset=['Model'])
# Normalize Model for consistency
df['Model'] = df['Model'].str.strip().str.upper()

# How many total unique models?
print("Total unique models:", df['Model'].nunique())

# Check if same model name appears under multiple makes
model_make_combo = df[['Make', 'Model']].drop_duplicates()
model_counts = df.groupby('Model')['Make'].nunique().sort_values(ascending=False)

# Show models that appear under multiple makes
print("Models used by multiple makes:\n", model_counts[model_counts > 1])
# Create a unique plane type identifier
df['Make_Model'] = df['Make'] + " - " + df['Model']

# Confirm uniqueness
print("Total unique Make-Model combinations:", df['Make_Model'].nunique())



Missing values in Model: 15
Total unique models: 3328
Models used by multiple makes:
 Model
CHALLENGER II    15
G-164B           15
S2R              13
G-164A           12
A36              12
                 ..
150L              2
767               2
150K              2
A-1C              2
SEAREY LSX        2
Name: Make, Length: 1039, dtype: int64
Total unique Make-Model combinations: 5007


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [361]:
columns_to_inspect = [
    'Engine.Type',
    'Weather.Condition',
    'Number.of.Engines',
    'Purpose.of.flight',
    'Broad.phase.of.flight'
]

# Show unique values and null counts
for col in columns_to_inspect:
    print(f"\nColumn: {col}")
    print("Missing:", df[col].isna().sum())
    print("Unique values:", df[col].unique())
# 1. Standardize case and whitespace
for col in ['Engine.Type', 'Weather.Condition', 'Purpose.of.flight', 'Broad.phase.of.flight']:
    df[col] = df[col].astype(str).str.strip().str.upper()

# 2. Collapse variants in Weather.Condition
df['Weather.Condition'] = df['Weather.Condition'].replace({
    'VMC': 'VFR',
    'VFR': 'VFR',
    'IMC': 'IFR',
    'IFR': 'IFR',
    'UNKNOWN': 'UNKNOWN'
})

# 3. Convert Number.of.Engines to numeric (some may be non-numeric strings)
df['Number.of.Engines'] = pd.to_numeric(df['Number.of.Engines'], errors='coerce')
# Confirm cleaning
for col in columns_to_inspect:
    print(f"\nCleaned Column: {col}")
    print(df[col].value_counts(dropna=False).head(10))



Column: Engine.Type
Missing: 3285
Unique values: ['Turbo Fan' 'Reciprocating' 'Turbo Prop' 'Turbo Jet' nan 'Unknown'
 'Turbo Shaft' 'Electric' 'Geared Turbofan' 'UNK']

Column: Weather.Condition
Missing: 2193
Unique values: ['VMC' 'IMC' 'UNK' nan 'Unk']

Column: Number.of.Engines
Missing: 1953
Unique values: [ 3.  4.  1.  2. nan  0.]

Column: Purpose.of.flight
Missing: 2854
Unique values: ['Unknown' nan 'Personal' 'Skydiving' 'Aerial Application' 'Positioning'
 'Instructional' 'Business' 'Public Aircraft' 'Ferry' 'Other Work Use'
 'Aerial Observation' 'Executive/corporate' 'Public Aircraft - Federal'
 'Air Race/show' 'Flight Test' 'Public Aircraft - State' 'Glider Tow'
 'Banner Tow' 'Firefighting' 'External Load' 'Air Race show'
 'Public Aircraft - Local' 'Air Drop' 'PUBS' 'ASHO']

Column: Broad.phase.of.flight
Missing: 17709
Unique values: ['Standing' 'Taxi' 'Cruise' 'Other' 'Climb' 'Takeoff' 'Landing' 'Approach'
 'Maneuvering' 'Descent' nan 'Go-around' 'Unknown']

Cleaned Column: En

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [363]:
# Get percentage of missing values per column
missing_percent = df.isna().mean().sort_values(ascending=False)

# Display columns with more than 50% missing data
print("Columns with > 50% missing values:\n")
print(missing_percent[missing_percent > 0.5])
# Drop columns with more than 50% missing values
columns_to_drop = missing_percent[missing_percent > 0.5].index
df.drop(columns=columns_to_drop, inplace=True)

# Confirm shape after removal
print("\nRemaining columns:", df.shape[1])


Columns with > 50% missing values:

Schedule       0.896921
Air.carrier    0.528400
dtype: float64

Remaining columns: 33


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [365]:
# Save the cleaned DataFrame to a new CSV file in the data folder
df.to_csv("data/cleaned_aviation_data.csv", index=False)

print(" Cleaned data saved to: data/cleaned_aviation_data.csv")
df = pd.read_csv("data/cleaned_aviation_data.csv")


 Cleaned data saved to: data/cleaned_aviation_data.csv
